# Introduction


For this notebook, I took inspiration from several sources:
* [Efficiently fine-tune Llama 3 with PyTorch FSDP and Q-Lora](https://www.philschmid.de/fsdp-qlora-llama3)  
* [Fine-tuning LLMs using LoRA](https://medium.com/@rajatsharma_33357/fine-tuning-llama-using-lora-fb3f48a557d5)  
* [Fine-tuning Llama-3–8B-Instruct QLORA using low cost resources](https://medium.com/@avishekpaul31/fine-tuning-llama-3-8b-instruct-qlora-using-low-cost-resources-89075e0dfa04)  
* [Llama2 Fine-Tuning with Low-Rank Adaptations (LoRA) on Gaudi 2 Processors](https://eduand-alvarez.medium.com/llama2-fine-tuning-with-low-rank-adaptations-lora-on-gaudi-2-processors-52cf1ee6ce11)  

# Install and import libraries

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl

In [2]:
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_key = user_secrets.get_secret("wandb_api")
import wandb
! wandb login $wandb_key

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
import os
import torch
from time import time
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)
from trl import SFTTrainer,setup_chat_format

2024-05-04 13:00:23.893392: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 13:00:23.893518: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 13:00:23.991146: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Initialize model

In [5]:
model_id = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

In [6]:
compute_dtype = torch.bfloat16
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True)

In [7]:
time_start = time()

model_config = AutoConfig.from_pretrained(
    model_id,
    trust_remote_code=True,
    max_new_tokens=1024
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_end = time()
print(f"Prepare model, tokenizer: {round(time_end-time_start, 3)} sec.")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Prepare model, tokenizer: 117.345 sec.


In [8]:
model, tokenizer = setup_chat_format(model, tokenizer)
model = prepare_model_for_kbit_training(model)

# Prepare the dataset

In [9]:
dataset_name = "HuggingFaceH4/ultrachat_200k"
dataset = load_dataset(dataset_name, split="test_sft")
dataset = dataset.shuffle(seed=42).select(range(10000))

def format_chat_template(row):
    chat = tokenizer.apply_chat_template(row["messages"], tokenize=False)
    return {"text":chat}

processed_dataset = dataset.map(
    format_chat_template,
    num_proc= os.cpu_count(),
)

dataset = processed_dataset.train_test_split(test_size=0.01)

Generating train_sft split:   0%|          | 0/207865 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/23110 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/256032 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/28304 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

# Training

## Training configuration

In [10]:
peft_config = LoraConfig(
        lora_alpha=64,
        lora_dropout=0.05,
        r=4,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",]
)

In [11]:
training_arguments = TrainingArguments(
        output_dir="./results_llama3_sft/",
        evaluation_strategy="steps",
        do_eval=True,
        optim="paged_adamw_8bit",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=2,
        per_device_eval_batch_size=8,
        log_level="debug",
        save_steps=1,
        logging_steps=1,
        learning_rate=8e-6,
        eval_steps=1,
        max_steps=10,
        num_train_epochs=10,
        warmup_steps=4,
        lr_scheduler_type="linear",
)

## Training process

In [12]:
os.environ["WANDB_DISABLED"] = "false"

In [13]:
trainer = SFTTrainer(
        model=model,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()

Map:   0%|          | 0/9900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

You have loaded a model on multiple GPUs. `is_model_parallel` attribute will be force-set to `True` to avoid any unexpected behavior such as device placement mismatching.
max_steps is given, it will override any value given in num_train_epochs
Currently training with a batch size of: 8
***** Running training *****
  Num examples = 9,900
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 10
  Number of trainable parameters = 10,485,760
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: gpreda. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240504_130343-zvykugxj
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run star-fleet-5
wandb: ⭐️ View project a

Step,Training Loss,Validation Loss
1,5.215200,5.378795
2,5.337100,5.338593
3,5.022100,5.261102
4,5.525300,5.150636
5,5.179000,5.015413
6,5.041100,4.912652
7,5.021300,4.836341
8,4.963700,4.781492
9,4.726800,4.745822
10,4.501000,4.728136


***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./results_llama3_sft/checkpoint-1
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /kaggle/input/llama-3/transformers/8b-chat-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(
tokenizer config file saved in ./results_llama3_sft/checkpoint-1/tokenizer_config.json
Special tokens file saved in ./results_llama3_sft/checkpoint-1/special_tokens_map.json
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
***** 

TrainOutput(global_step=10, training_loss=5.053261232376099, metrics={'train_runtime': 3256.7505, 'train_samples_per_second': 0.049, 'train_steps_per_second': 0.003, 'total_flos': 3693978562068480.0, 'train_loss': 5.053261232376099, 'epoch': 0.01615508885298869})

# Conclusions

By reducing the batch_size, max_seq_length and the LoRA rank, we managed to run the Llama3 fine-tunning with QLoRA in the Kaggle environment.